In [23]:
import pandas as pd
import requests
import json

In [39]:
ICUBeds = pd.read_csv("ICUsByCounty.csv") 

In [40]:
ICUBeds.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed
0,Alabama,Autauga,6,55036,10523,19.1,1754.0
1,Alabama,Baldwin,51,203360,53519,26.3,1049.0
2,Alabama,Barbour,5,26201,6150,23.5,1230.0
3,Alabama,Bibb,0,22580,4773,21.1,NaN
4,Alabama,Blount,6,57667,13600,23.6,2267.0


In [152]:
api_response = requests.get('https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000')
hospitals = pd.json_normalize(api_response.json()['features'])
df_length = len(hospitals)
api_target = 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000'

while True:
    api_response = requests.get(api_target+'&resultOffset='+str(df_length))
    new = pd.json_normalize(api_response.json()['features'])
    hospitals = hospitals.append(new)
    if (df_length == len(hospitals)):
        break
    df_length = len(hospitals)

In [155]:
hospitals = hospitals.rename(columns = {'attributes.BEDS': 'Hospital Beds', 'attributes.COUNTY': 'County'})

In [263]:
hospitals = hospitals.assign(sanitized=lambda df: df['Hospital Beds'].abs())

In [266]:
hospitals['Hospital Beds'] = hospitals['sanitized']
hospitals['Hospital Beds'] = hospitals['Hospital Beds'].astype(int)

In [267]:
beds = hospitals[['County', 'Hospital Beds']].groupby('County').sum()

In [268]:
beds = beds.assign(lowercase=lambda df: df.index.str.lower())
ICUBeds = ICUBeds.assign(lowercase=lambda df: df.County.str.lower())

In [269]:
result = ICUBeds.merge(beds, how='left', on='lowercase')

In [270]:
result = result.drop('lowercase', axis=1)

In [245]:
result.to_csv('HospitalAndICUBeds.csv')

In [271]:
result.loc[result['Hospital Beds'] > 0]

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,Hospital Beds
0,Alabama,Autauga,6,55036,10523,19.1,1754.0,85.0
1,Alabama,Baldwin,51,203360,53519,26.3,1049.0,738.0
2,Alabama,Barbour,5,26201,6150,23.5,1230.0,146.0
3,Alabama,Bibb,0,22580,4773,21.1,NaN,1329.0
4,Alabama,Blount,6,57667,13600,23.6,2267.0,511.0
...,...,...,...,...,...,...,...,...
3137,Wyoming,Sweetwater,10,44527,7015,15.8,702.0,115.0
3138,Wyoming,Teton,6,22923,4298,18.7,716.0,101.0
3139,Wyoming,Uinta,6,20758,3554,17.1,592.0,225.0
3140,Wyoming,Washakie,0,8253,2206,26.7,NaN,18.0
